In [54]:
import pandas as pd
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
train1 = pd.read_csv(r"C:\Users\Admin\Documents\GitHub\Spooky-author-identification\train.csv")#19579
test = pd.read_csv(r"C:\Users\Admin\Documents\GitHub\Spooky-author-identification\test.csv")#8392
submit = pd.read_csv(r"C:\Users\Admin\Documents\GitHub\Spooky-author-identification\sample_submission.csv")

In [55]:
from math import floor
valid= train1.loc[:floor((train1.shape[0]*1)/3)]
train = train1.loc[floor((train1.shape[0]*1)/3):]
#train.head()


In [56]:
print train.shape[0]

13053


In [57]:
#test.head()

In [58]:
import re
from nltk.corpus import stopwords 
import string
from nltk.stem import PorterStemmer
ps = PorterStemmer()
stop = set(stopwords.words('english'))# such as a, an, and, are,
exclude = set(string.punctuation) # such as: , . "" {}
"""
#tokenize text
def token(doc):
    split = " ".join(i for i in doc.lower().split())
    words = re.findall(r"[\w']+", split)
    stop_free = " ".join([i for i in words if i not in stop]) # no stop word
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude) # no punctuation 
    token_a = re.findall(r"[\w']+", punc_free)
    token_b =[ps.stem(a) for a in token_a]
    token_c = " ".join([b for b in token_b])
    return token_c
train["token"] = [token(a) for a in train["text"]]
valid["token"]= [token(a) for a in valid["text"]]
test["token"] = [token(a) for a in test["text"]]"""
    

'\n#tokenize text\ndef token(doc):\n    split = " ".join(i for i in doc.lower().split())\n    words = re.findall(r"[\\w\']+", split)\n    stop_free = " ".join([i for i in words if i not in stop]) # no stop word\n    punc_free = \'\'.join(ch for ch in stop_free if ch not in exclude) # no punctuation \n    token_a = re.findall(r"[\\w\']+", punc_free)\n    token_b =[ps.stem(a) for a in token_a]\n    token_c = " ".join([b for b in token_b])\n    return token_c\ntrain["token"] = [token(a) for a in train["text"]]\nvalid["token"]= [token(a) for a in valid["text"]]\ntest["token"] = [token(a) for a in test["text"]]'

In [59]:
#dividing the training set into groups
#hpl = train[train["author"] == "HPL"][['id','token']]
#mws = train[train["author"] == "MWS"][['id','token']]
#eap = train[train["author"] == "eap"][['id','token']]

## Simple Naive Bayesian Model 

In [61]:
# TFIDF model
tfidf = TfidfTransformer()
#vector count model
vector= CountVectorizer(decode_error = 'ignore',stop_words = 'english')
# fit and transform the training set
X_train = vector.fit_transform(list(train['text']))
X_train_tfidf = tfidf.fit_transform(X_train)
y_train = list(train['author'])
#transform the testing set with the training model
X_test = vector.transform(list(valid['text']))
X_test_tfidf= tfidf.transform(X_test)
y_test = list(valid['author'])

In [62]:
print X_train.shape
print len(y_train)
print X_test.shape
print len(y_test)

(13053, 21147)
13053
(6527, 21147)
6527


In [88]:
clf = MultinomialNB().fit(X_train, y_train)
predicted = clf.predict(X_test_tfidf)
float(np.mean(predicted == y_test))

0.821357438333078

## Support Vector Machine

In [81]:

text_clf_svm = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()), ('clf_svm', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, n_iter=5, random_state=42))])

In [89]:
_= text_clf_svm.fit(list(train['text']), list(train['author']))
svm_predicted = text_clf_svm.predict(list(valid['text']))
float(np.mean(svm_predicted == y_test))

0.7464378734487513